In [59]:
# Begin by importing some necessary modules
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes

#Helper function that returns the number of characters different in two strings
def char_diff(str1, str2):
    return sum ( str1[i] != str2[i] for i in range(len(str1)) )

# Messages to be hashed
message_1 = b"Buy 10000 shares of WXYZ stock now!"
message_2 = b"Buy 10000 shares of VXYZ stock now!"

print(f"The two messages differ by { char_diff(message_1, message_2)} characters")

The two messages differ by 1 characters


In [60]:
# Create new SHA-256 hash objects, one for each message
chf_1 = hashes.Hash(hashes.SHA256(), backend=default_backend())
chf_2 = hashes.Hash(hashes.SHA256(), backend=default_backend())

# Update each hash object with the bytes of the corresponding message
chf_1.update(message_1)
chf_2.update(message_2)

# Finalize the hash process and obtain the digests
digest_1 = chf_1.finalize()
digest_2 = chf_2.finalize()

#Convert the resulting hash to hexadecimal strings for convenient printing
digest_1_str = digest_1.hex()
digest_2_str = digest_2.hex()

#Print out the digests as strings 
print(f"digest-1: {digest_1_str}")
print(f"digest-2: {digest_2_str}")

print(f"The two digests differ by { char_diff(digest_1_str, digest_2_str)} characters")

digest-1: 6e0e6261b7131bd80ffdb2a4d42f9d042636350e45e184b92fcbcc9646eaf1e7
digest-2: 6b0abb368c3a1730f935b68105e3f3ae7fd43d7e786d3ed3503dbb45c74ada46
The two digests differ by 57 characters


In [61]:
# Install the library if needed
%pip install secretpy

# import the required crypto functions which will be demonstrated later
from secretpy import Caesar
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from functools import reduce
import numpy as np

# Set the plaintext we want to encrypt
plaintext=u"this is a strict top secret message for intended recipients only"
print(f"\nGiven plaintext: {plaintext}")

Note: you may need to restart the kernel to use updated packages.

Given plaintext: this is a strict top secret message for intended recipients only


In [62]:

# initialize the required python object for doing Caesar shift encryption
caesar_cipher = Caesar()

# Define the shift, ie the key
caesar_key = 5 
print(f"Caesar shift secret key: {caesar_key}")

# Define the alphabet
alphabet=('a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ')
print(f"alphabet: {alphabet}")

Caesar shift secret key: 5
alphabet: ('a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ')


In [63]:

caeser_ciphertext = caesar_cipher.encrypt(plaintext, caesar_key, alphabet)
print(f"Encrypted caeser shift ciphertext: {caeser_ciphertext}")

Encrypted caeser shift ciphertext: ymnxenxefexywnhyeytuexjhwjyerjxxfljektwensyjsijiewjhnunjsyxetsqc


In [64]:
caeser_plaintext = caesar_cipher.decrypt(caeser_ciphertext, caesar_key, alphabet)
print(f"Decrypted caeser shift plaintext: {caeser_plaintext}\n")

Decrypted caeser shift plaintext: this is a strict top secret message for intended recipients only



In [65]:
# lamba defines an inline function in this case that takes two values a,b with the resulting expression of a+b
# reduce uses a two-argument function(above), and applies this to all the entries in the list (random alphabet characters) cumulatively
aes_key = reduce(lambda a, b: a + b, [np.random.choice(alphabet) for i in range(16)])

print(f'AES secret key: {aes_key}')

AES secret key: kmvalqfrkiaatzkp


In [66]:
aes_initialization_vector = reduce(lambda a, b: a + b, [np.random.choice(alphabet) for i in range(16)])
print(f"AES initialization vector: {aes_initialization_vector}")

AES initialization vector: gwwo ofukcqf mgk


In [67]:
# The encryptor is setup using the key & CBC. In both cases we need to convert the string (utf-8) into bytes
sender_aes_cipher = Cipher(algorithms.AES(bytes(aes_key, 'utf-8')), modes.CBC(bytes(aes_initialization_vector, 'utf-8')))
aes_encryptor = sender_aes_cipher.encryptor()

# update can add text to encypt in chunks, and then finalize is needed to complete the encryption process
aes_ciphertext = aes_encryptor.update(bytes(plaintext, 'utf-8')) + aes_encryptor.finalize()

# Note the output is a string of bytes
print(f"Encrypted AES ciphertext: {aes_ciphertext}")

Encrypted AES ciphertext: b'8\x931\xc4\xa8\xa9\xc4\x04\xc5\xeb\xd4\xe7$\x85\xf6\xbe5\\X\xc1f\x00|\xa4\xc5\xafz/ZM\x8bAH;\xec x\x83\x97\x9e\xb7-\x90)\x13\x19ypNM#\x84\xeb@\x97T\xb4\x18\xf5\xc4\xe6\x0f\xc9\xdd'


In [68]:
# Similar setup of AES to what we did for encryption, but this time, for decryption
receiver_aes_cipher = Cipher(algorithms.AES(bytes(aes_key, 'utf-8')), modes.CBC(bytes(aes_initialization_vector, 'utf-8')))
aes_decryptor = receiver_aes_cipher.decryptor()

# Do the decryption
aes_plaintext_bytes = aes_decryptor.update(aes_ciphertext) + aes_decryptor.finalize()

# convert back to a character string (we assume utf-8)
aes_plaintext = aes_plaintext_bytes.decode('utf-8')

print(f"Decrypted AES plaintext: {aes_plaintext}")

Decrypted AES plaintext: this is a strict top secret message for intended recipients only


# Quantum Safe Cryptography
- Learning with error
Illustration of LWE encryption in Python
The following simple example shows the use of LWE for encryption and decryption. Bob will send an encrypted message to Alice.

First, Alice and Bob agree on the problem parameters. These are explained in detail in the maths section above, but in summary we require 
n
,
q
,
N
,
χ
n,q,N,χ.

We start with some of the basic parameters
- N represents the number of samples
- q is a modulus
- n is known as the security parameter, or vector dimension.

These parameters are all public information in principle.

In [69]:
import numpy as np
from matplotlib import pyplot as plt
n=8
q=127
N=int(1.1*n*np.log(q))
sigma=1.0
print(f"n={n},q={q},N={N},sigma={sigma}")

n=8,q=127,N=42,sigma=1.0


We also need a way of introducing errors

Here χ represent the errors we want to introduce - we use a discrete Gaussian distribution on 
Zq ​characterized by mean 0 and standard deviation σ.

In [70]:
def chi(stdev, modulus):
    return round((np.random.randn() * stdev**2))%modulus

# print some examples
sd=2
m=1000
for x in range(10):
  print("chi = ",chi(sd,m))

chi =  3
chi =  5
chi =  1
chi =  0
chi =  998
chi =  1
chi =  0
chi =  993
chi =  992
chi =  998


In [71]:
#Alice's private key
alice_private_key = np.random.randint(0, high=q, size=n)
print(f"Alice's private key: {alice_private_key}")

Alice's private key: [ 25   5 104  30  32   4 106  22]


In [72]:
#Alice's Public Key
alice_public_key = []

# N is the number of values we want in the key
for i in range(N):
    # Get n random values between 0 and <q
    a = np.random.randint(0, high=q, size=n)
    # get an error to introduce
    epsilon = chi(sigma, q)
    #  calculate dot product (ie like array multiplication)
    b = (np.dot(a, alice_private_key) + epsilon) % q
    # value to be added to the key -
    sample = (a, b)
    alice_public_key.append(sample)
    
print(f"Alice's public key: {alice_public_key}")

Alice's public key: [(array([ 20,  88,  65,  60, 109,  26,  81,  81]), 93), (array([104,  63,  41,  81,  81,  21,  13, 102]), 31), (array([ 11,  67,  83, 123, 103,  53,  45,  15]), 75), (array([ 83,  23,  31,  28,  52, 126,  61,  56]), 118), (array([107, 110,  58, 112,  38,  85,   6,  43]), 8), (array([108, 108,  33,  38,  63,  65, 122,  86]), 19), (array([ 79,  15,  50,  98,  89,  76, 100,  92]), 60), (array([ 23,  88, 112,  36,   0,   3,  63,  64]), 123), (array([ 86, 103,  62, 103,  98, 102, 121,  42]), 31), (array([ 35,  24,  82,  66,  69, 107,  96, 125]), 14), (array([76, 11,  4, 56, 27, 99, 21, 41]), 57), (array([ 25,  34, 107,  56, 106,  37,  45,   8]), 119), (array([113, 114, 100,  42,  19,  75,  18,  92]), 83), (array([ 46,  74,  57, 123,  53, 110,  13,  54]), 92), (array([ 54,  60,   6,  98,  40, 124,  97,  29]), 3), (array([47, 19, 13, 99, 64, 40, 18, 91]), 27), (array([126, 120,  32,   8, 120,  87, 103,  67]), 22), (array([ 28, 112,  74,  77, 119,  23,  85,  38]), 121), (ar

In [73]:
#Encryption
bob_message_bit = 1
print(f"Bob's message bit={bob_message_bit}")

Bob's message bit=1


In [74]:
# a list of N values between 0 and <2 - ie 0 or 1
r = np.random.randint(0, 2, N)
print(r)

[1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 0 1 0 0 1 0 0 1 0 0 0 1 1 0 1 1 0 0 0 1 0
 1 0 0 0 0]


In [75]:
sum_ai=np.zeros(n, dtype=int)
sum_bi=0

for i in range(N):
    sum_ai = sum_ai + r[i] * alice_public_key[i][0]
    sum_bi = sum_bi + r[i] * alice_public_key[i][1]
sum_ai = [ x % q for x in sum_ai ]
# sum_bi = sum_bi 
ciphertext = (sum_ai, (bob_message_bit*int(np.floor(q/2))+sum_bi)%q)
print(f"ciphertext is: {ciphertext}")

ciphertext is: ([67, 98, 9, 0, 43, 48, 119, 30], 94)


In [76]:
#Decryption
adots = np.dot(ciphertext[0], alice_private_key) % q
b_adots = (ciphertext[1] - adots) % q

decrypted_message_bit = round((2*b_adots)/q) % 2

print(f"original message bit={bob_message_bit}, decrypted message bit={decrypted_message_bit}")

assert bob_message_bit == decrypted_message_bit

original message bit=1, decrypted message bit=1


In [77]:
bob_message_bits = np.random.randint(0, 2, 16)
print(f"Bob's message bits are : {bob_message_bits}")
decrypted_bits = []

for ib in range(len(bob_message_bits)):
    bob_message_bit = bob_message_bits[ib]

    r = np.random.randint(0, 2, N)
    
    sum_ai=np.zeros(n, dtype=int)
    sum_bi=0
    for i in range(N):
        sum_ai = sum_ai + r[i] * alice_public_key[i][0]
        sum_bi = sum_bi + r[i] * alice_public_key[i][1]
    sum_ai = [ x % q for x in sum_ai ]

    ciphertext = (sum_ai, (bob_message_bit*int(np.floor(q/2))+sum_bi)%q)

    adots = np.dot(ciphertext[0], alice_private_key) % q
    b_adots = (ciphertext[1] - adots) % q

    decrypted_message_bit = round((2*b_adots)/q) % 2
    assert decrypted_message_bit == bob_message_bit

    decrypted_bits.append(decrypted_message_bit)
    
print(f"Decrypted message bits = {np.array(decrypted_bits)}")

Bob's message bits are : [1 0 0 0 0 1 1 1 0 1 1 0 1 1 1 1]
Decrypted message bits = [1 0 0 0 0 1 1 1 0 1 1 0 1 1 1 1]


In [79]:
import warnings
warnings.filterwarnings('ignore')
import oqs
from pprint import pprint

In [82]:
kems = oqs.
kems = oqs.get_enabled_kem_mechanisms()
print(kems, compact=True)

AttributeError: module 'oqs' has no attribute 'get_enabled_kem_mechanisms'